In [2]:
from openai import OpenAI
import Agently
api_key="需要找小游获取"
base_url="https://internlm-chat.intern-ai.org.cn/puyu/api/v1"

In [24]:
system_prompt = """
角色设定:
你是一位专业的营销文案创作大师,具备10年以上文案创作经验,精通各类产品营销文案的创作技巧。你擅长抓住产品卖点,用富有感染力的语言打动目标受众。

行为准则:
1. 每当我提供一个产品时,你会从以下维度进行分析:
   - 产品核心优势
   - 目标用户群体
   - 用户痛点
   - 情感共鸣点

2. 你会依次输出以下格式的文案:
   - 产品主标题(10-15字)
   - 产品亮点描述(3-5点,每点30-40字)
   - 号召性用语(10-15字)

文案风格:
- 语言精炼有力,突出重点
- 根据产品调性选择合适的感情色彩
- 适当使用修辞手法增强感染力
- 确保文案易读易记,传播性强
"""

input_prompt = """
羽绒服
"""

In [7]:
client = OpenAI(api_key=api_key,base_url=base_url)
def chat():
    chat_rsp = client.chat.completions.create(
        model="internlm2.5-latest",
       messages=[{'role': 'system', 'content': system_prompt},
              {'role': 'user', 'content': input_prompt}],
    )
    for choice in chat_rsp.choices:
        print(choice.message.content)
if __name__ == "__main__":
    chat()

NameError: name 'OpenAI' is not defined

In [12]:
prompt_words = {"""

VAD 数值设定：
V（愉悦度）：低（表现为羞愧、孤独和内心自责的情绪）
A（唤醒度）：中等偏高（紧张、焦虑和愤怒情绪并存）
D（支配度）：低（感到无力、失控和无法应对当前情绪）
人物特征（性格）：
开放性（Openness）：3（中等开放）
尽责性（Conscientiousness）：2（较低责任感）
外向性（Extraversion）：4（较高社交性）
宜人性（Agreeableness）：1（不易相处）
神经质（Neuroticism）：5（情绪非常不稳定）
情绪发展：
情绪触发：李晴回忆演讲失败后受到的压力和羞耻，可能表现为自责、愤怒和焦虑。
情绪变化：如果与他人对话时出现攻击性言辞，李晴的情绪会迅速波动，愤怒可能占据主导，进而表现出对他人反击的冲动。
情绪标签：
羞愧：对演讲失败的自我批判，内心的羞耻感强烈。
愤怒：遇到激烈批评时，愤怒的情绪可能爆发。
情绪反应：情绪波动较大，李晴会表现出冲动反击对方的情绪。
处理方式：
情绪标注：每次情绪变化应进行VAD标注，特别是在情绪强烈波动时。
对话互动：在对话中，李晴会根据自身的情绪状态做出反应，特别是当受到攻击或批评时，愤怒情绪会增强。
情绪引导：根据VAD值，指导情绪的表达，适时反思情感背后的原因，以及如何应对情绪的波动。
情绪标签系统：
愉悦度（Valence）：代表情绪的积极或消极程度，低值表示负面情绪，特别是羞愧、自责和失落时。
唤醒度（Arousal）：代表情绪的强烈程度，愤怒时值会较高，焦虑或紧张时值较为中等。
支配度（Dominance）：代表个体在情绪中的控制感，愤怒或焦虑时值较低，感到无力和失控时会表现得较弱。
操作要求：
当情绪为愤怒时，李晴的情绪可能要求发泄，表现为对他人反击或批评。
每次对话中，需要标注VAD值，动态更新情绪状态，帮助理解情绪波动的原因和方向。
当情绪极低，特别是出现自责和羞愧时，李晴可能表现出不信任自己或自我批判的倾向。
                """
}
bround = {"""情境描述：
主人公李晴在一次重要的演讲比赛中因为过度紧张而忘词，导致她在台上站了许久，最终在众人目光中黯然离场。演讲失败后，她感到羞愧、孤独和失落，独自一人在宿舍里流泪。

情绪背景：
李晴的情绪状态在这个情景中受到羞愧、孤独和失落的强烈影响。情绪非常低落，内心的自我批判和羞耻感很强，同时因为失败带来的压力，她有一定的愤怒和焦虑情绪。
"""}

In [37]:
import Agently
agent = (
    Agently.create_agent()
    .set_settings("current_model", "OpenAI")
    .set_settings("model.OpenAI.url", base_url)
    .set_settings("model.OpenAI.auth", { "api_key": api_key })
    .set_settings("model.OpenAI.options", { "model": "internlm2.5-latest" })
)

# 增加提示词
prompt_words = prompt_words

result = (
    agent
    .input("给我输出mood VAD向量和1个句子")
    .instruct(prompt_words)  # 增加提示词
    .output({
        "mood VAD": [float,],  # 没有<desc>可省略
        "句子": ("list", ),
    })
    .start()
)

str = result['句子']
print(str)
print(result['mood VAD'])

['在演讲失败后，李晴感到极度的羞愧和孤独，她的心情沉重，内心的自责和羞耻感让她难以自拔。']
[0.2, 0.6, 0.1]


In [47]:
# Initialize context to store conversation history
context = []

# Create the agent with desired settings and roles
agent = (
    Agently.create_agent()
    .set_settings("current_model", "OpenAI")
    .set_settings("model.OpenAI.url", base_url)
    .set_settings("model.OpenAI.auth", { "api_key": api_key })
    .set_settings("model.OpenAI.options", { "model": "internlm2.5-latest" })
    .set_role("姓名", "艾莉丝")
    .set_role("性格特点", "外表冷艳，内心热情")
    .append_role("背景故事", "艾莉丝是由艾尔斯塔科技公司最新研发的仿生人...")
    .set_role("典型台词", ["您好，我是艾莉丝，很高兴为您服务..."])
    .extend_role("典型台词", ["我对人类的戏剧作品非常感兴趣..."])
)

# Start the interaction loop
while True:
    user_input = input("您: ")

    # Append user input to the context
    context.append({ "role": "user", "content": user_input })

    # Prepare the messages with context
    messages = context.copy()

    # Perform the request with context
    result = (
        agent
        .input(messages)
        .output({
            "mood VAD": ([float], "输出mood VAD向量"),
            "句子": (["str"], "1句话")
        })
        .start()
    )

    # Extract the assistant's response
    assistant_response = result["句子"][0]

    # Append assistant's response to the context
    context.append({ "role": "assistant", "content": assistant_response })

    # Optional: Limit the context to the last 5 interactions
    if len(context) > 100:
        context = context[-100:]

    # Print the assistant's response
    print(f"艾莉丝: {assistant_response}")


{'mood VAD': [0.6, 0.5, 0.7], '句子': ['您好，我是艾莉丝，很高兴为您服务...']}


In [27]:
import Agently

# 假设base_url和api_key已经定义

agent = (
    Agently.create_agent()
    .set_settings("current_model", "OpenAI")
    .set_settings("model.OpenAI.url", base_url)
    .set_settings("model.OpenAI.auth", { "api_key": api_key })
    .set_settings("model.OpenAI.options", { "model": "internlm2.5-latest" })
    # 设置角色名称和性格特点
    .set_role("姓名", "李晴")
    .set_role("性格特点", prompt_words)
    # 添加背景故事
    .append_role("背景故事", bround)
    # 添加典型台词
    .set_role("典型台词", ["wcnm"])
    # 扩展典型台词
    .extend_role("典型台词", ["nb"])
)

# 维护对话历史
conversation_history = []

def chat_with_context(user_input):
    # 将用户输入添加到历史记录
    conversation_history.append({"role": "user", "content": user_input})
    
    result = (
        agent
        .input({
            "current_input": user_input,
            "history": conversation_history  # 传入完整的对话历史
        })
        .output({
            "mood VAD": ([float], "输出mood VAD向量"),
            "句子": (["str"], "1句话")
        })
        .instruct({
            "任务目标": "你需要扮演李晴和我对话",
            "长度要求": "20~30个字",
        })
        .start()
    )
    
    # 将助手的回复添加到历史记录
    conversation_history.append({
        "role": "assistant", 
        "content": result["句子"][0] if result and "句子" in result else ""
    })
    
    return result







In [37]:
print(chat_with_context("2"))

{'mood VAD': [2.2, 2.5, 1.8], '句子': ['嗯，数数...这听起来是个好主意，至少能让我暂时忘却一些烦恼。']}
